# Трансформер

## Импорт библиотек

In [3]:
import torch
import evaluate
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, pipeline, default_data_collator
from datasets import load_dataset

MODEL_NAME = "DeepPavlov/rubert-base-cased"

## Использование предобученной модели

In [4]:
from transformers import pipeline


question_answerer = pipeline("question-answering", model = MODEL_NAME)

question = "Вследствие чего климат Каринтии нетипично суров?"
context = "Климат Каринтии, вообще говоря, довольно суровый для той широты, в которой она находится, вследствие высокого положения страны над уровнем моря, её горных хребтов, отчасти закрывающих её от теплых воздушных течений, и, наконец, её ледников. Особенно суров (сравнительно) климат Верхней Каринтии (северная и северо-западная часть региона); в Нижней Каринтии (восточная и юго-восточные области) климат значительно мягче, и совсем уже мягким теплым климатом отличается долина реки Лафант, где растут средиземноморские плоды и овощи. В широких долинах и котловинах наблюдается сравнительно холодная зима с продолжительным снежным покровом и довольно теплое лето. Дождя и снега выпадает много, особенно в горах; среднее количество более 95 см в год. Летние осадки преобладают."
print(question_answerer(question = question, context = context))

question = "Когда у озимых однолетников прорастают семена?"
context = "Однолетние травянистые растения отмирают полностью в конце вегетационного периода, либо после завершения цветения и плодоношения, а затем они снова вырастают из семян. Однолетники полностью проходят свой жизненный цикл за один сезон, в течение которого они вырастают из семян, цветут и после цветения и плодоношения отмирают. У яровых однолетников семена прорастают весной, и в это же лето растения после плодоношения отмирают. У озимых однолетников семена прорастают осенью, растения зимуют обычно в виде укороченного побега с розеткой прикорневых листьев, а в следующем году цветут, плодоносят и отмирают."
print(question_answerer(question = question, context = context))

question = "Где в древности применялись и земноводные, и галлюциногенные грибы?"
context = "Ядовитый гриб , поганка а иногда и любой гриб по-английски — toadstool, буквально скамейка жаб ; аналогичные термины есть в нидерландском и немецком языках: нидерл. padde(n)stoel, нем. Krötenschwamm (букв. жабья губка ), связанные с жабой названия ядовитых грибов есть и в других европейских языках. О происхождении слова toadstool есть два предположения: 1) сравнение с ядовитыми (или считавшимися в древности ядовитыми) жабами, 2) фоно-семантическое соответствие с немецким Todesstuhl — смертельный стул . По предположению этномиколога Р. Уоссона[8], сравнение с жабами возникло из-за того, что в древности и земноводные, и галлюциногенные грибы применялись в колдовских языческих обрядах[6][9]"
print(question_answerer(question = question, context = context))

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


{'score': 0.00023675330157857388, 'start': 682, 'end': 751, 'answer': 'много, особенно в горах; среднее количество более 95 см в год. Летние'}
{'score': 0.00021647482935804874, 'start': 158, 'end': 179, 'answer': 'из семян. Однолетники'}
{'score': 0.00014021845709066838, 'start': 565, 'end': 608, 'answer': 'жабами возникло из-за того, что в древности'}


## Определение класса QADatasetProcessor для обработки данных

In [5]:
class QADatasetProcessor:
    """
    Класс для загрузки, токенизации и предобработки данных для задачи вопрос-ответ (QA).
    Использует SQuAD-подобный датасет SberQuad.
    """

    def __init__(self, model_name: str = MODEL_NAME):
        """
        Инициализирует токенизатор на основе заданной модели.

        Параметры:
            model_name (str): название модели, совместимой с Hugging Face.
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def load_data(self):
        """
        Загружает датасет SberQuad из Hugging Face Datasets.

        Возвращает:
            DatasetDict: словарь с разбивкой на train и validation.
        """
        self.dataset = load_dataset("kuznetsoffandrey/sberquad")
        return self.dataset

    def _preprocess_examples(self, examples):
        """
        Токенизирует и преобразует примеры в формат, совместимый с моделью вопрос-ответ.

        Параметры:
            examples (dict): батч примеров из датасета.

        Возвращает:
            dict: словарь с токенами и позициями начала и конца ответов.
        """
        questions = examples["question"]
        contexts = examples["context"]
        answers = examples["answers"]

        answer_starts = [ans["answer_start"][0] for ans in answers]
        answer_texts = [ans["text"][0] for ans in answers]

        inputs = self.tokenizer(
            questions,
            contexts,
            max_length=384,
            truncation="only_second",
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        offset_mapping = inputs.pop("offset_mapping")
        overflow_to_sample_mapping = inputs.pop("overflow_to_sample_mapping")

        start_positions = []
        end_positions = []

        for i, offsets in enumerate(offset_mapping):
            sample_idx = overflow_to_sample_mapping[i]
            start_char = answer_starts[sample_idx]
            end_char = start_char + len(answer_texts[sample_idx])

            sequence_ids = inputs.sequence_ids(i)
            context_start = sequence_ids.index(1)
            context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

            if not (
                offsets[context_start][0]
                <= start_char
                < end_char
                <= offsets[context_end][1]
            ):
                start_positions.append(0)
                end_positions.append(0)
                continue

            token_start = context_start
            while token_start <= context_end and offsets[token_start][0] <= start_char:
                token_start += 1
            start_positions.append(token_start - 1)

            token_end = context_end
            while token_end >= context_start and offsets[token_end][1] >= end_char:
                token_end -= 1
            end_positions.append(token_end + 1)

        inputs["start_positions"] = start_positions
        inputs["end_positions"] = end_positions
        inputs["overflow_to_sample_mapping"] = overflow_to_sample_mapping
        return inputs

    def get_tokenized_dataset(self, dataset):
        """
        Применяет токенизацию ко всему датасету.

        Параметры:
            dataset (DatasetDict): оригинальный датасет SberQuad.

        Возвращает:
            DatasetDict: токенизированный датасет.
        """
        return dataset.map(
            self._preprocess_examples,
            batched=True,
            remove_columns=dataset["train"].column_names,
            batch_size=100,
        )

## Определение класса QAModelTrainer для обучения модели

In [6]:
class QAModelTrainer:
    """
    Класс для обучения модели на задаче вопрос-ответ с использованием Trainer API.
    """

    def __init__(
        self, model_name: str = MODEL_NAME, tokenizer=None, training_args: dict = None
    ):
        """
        Инициализация модели и аргументов тренировки.

        Параметры:
            model_name (str): имя модели.
            tokenizer: токенизатор, используемый в pipeline.
            training_args (dict): словарь с параметрами обучения.
        """
        self.model = AutoModelForQuestionAnswering.from_pretrained(model_name)
        self.tokenizer = tokenizer
        self.training_args = training_args or {
            "output_dir": "./results",
            "learning_rate": 2e-5,
            "per_device_train_batch_size": 8,
            "num_train_epochs": 3,
            "weight_decay": 0.01,
            "eval_strategy": "epoch",
            "save_strategy": "epoch",
            "logging_dir": "./logs",
            "fp16": torch.cuda.is_available(),
        }

    def setup_trainer(self, tokenized_dataset, original_dataset):
        """
        Создаёт Trainer с метриками, моделью и параметрами.

        Параметры:
            tokenized_dataset (DatasetDict): токенизированный датасет.
            original_dataset (DatasetDict): исходный SberQuad.

        Возвращает:
            Trainer: объект Trainer.
        """
        args = TrainingArguments(**self.training_args)
        squad_metric = evaluate.load("squad_v2")

        def compute_metrics(p):
            """Вычисляет метрики точности для задачи QA."""
            start_logits, end_logits = p.predictions
            start_pred = torch.argmax(torch.tensor(start_logits), dim=1).numpy()
            end_pred = torch.argmax(torch.tensor(end_logits), dim=1).numpy()

            formatted_predictions = []
            references = []

            for i in range(len(start_pred)):
                sample_idx = tokenized_dataset["validation"][i][
                    "overflow_to_sample_mapping"
                ]
                original_sample = original_dataset["validation"][sample_idx]

                prediction_text = self.tokenizer.decode(
                    tokenized_dataset["validation"][i]["input_ids"][
                        start_pred[i] : end_pred[i] + 1
                    ],
                    skip_special_tokens=True,
                )

                references.append(
                    {
                        "id": str(original_sample["id"]),
                        "answers": {
                            "text": original_sample["answers"]["text"],
                            "answer_start": original_sample["answers"]["answer_start"],
                        },
                    }
                )

                formatted_predictions.append(
                    {
                        "id": str(original_sample["id"]),
                        "prediction_text": prediction_text,
                        "no_answer_probability": 0.0,
                    }
                )

            return squad_metric.compute(
                predictions=formatted_predictions, references=references
            )

        self.trainer = Trainer(
            model=self.model,
            args=args,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset["validation"],
            data_collator=default_data_collator,
            compute_metrics=compute_metrics,
        )
        return self.trainer

    def train(self):
        """Запуск обучения модели."""
        return self.trainer.train()

    def save_model(self, path: str = "./qa_model"):
        """
        Сохраняет модель и токенизатор в указанную директорию.

        Параметры:
            path (str): путь для сохранения.
        """
        self.model.save_pretrained(path)
        self.tokenizer.save_pretrained(path)

## Определение класса QAPipeline для использования модели

In [7]:
class QAPipeline:
    """
    Класс для выполнения предсказаний с помощью обученной модели в формате question-answering pipeline.
    """

    def __init__(self, model_path: str = "./qa_model"):
        """
        Инициализирует модель и токенизатор из указанного пути.

        Параметры:
            model_path (str): путь до директории с моделью.
        """
        self.model = AutoModelForQuestionAnswering.from_pretrained(model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.pipeline = pipeline(
            "question-answering",
            model = self.model,
            tokenizer = self.tokenizer,
            device = 0 if torch.cuda.is_available() else -1,
        )

    def predict(self, context: str, question: str):
        """
        Предсказывает ответ на вопрос по заданному контексту.

        Параметры:
            context (str): текстовый контекст.
            question (str): вопрос к контексту.

        Возвращает:
            dict: словарь с ответом и оценкой.
        """
        return self.pipeline(
            question=question,
            context=context,
            max_seq_len=384,
            doc_stride=128,
            handle_impossible_answer=True,
        )

## Инициализация и загрузка данных

In [8]:
processor = QADatasetProcessor()
dataset = processor.load_data()

print("Структура датасета:", end = '\n\n')
print(dataset)
print(f"Количество записей в train датасете: {len(dataset['train'])}")
print(f"Количество записей в validation датасете: {len(dataset['validation'])}")
print(f"Количество записей в test датасете: {len(dataset['test'])}")
print(f"Общее количество записей: {len(dataset['test']) + len(dataset['train']) + len(dataset['validation'])}")


Структура датасета:

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})
Количество записей в train датасете: 45328
Количество записей в validation датасете: 5036
Количество записей в test датасете: 23936
Общее количество записей: 74300


## Проверка объема и качества данных

In [11]:
def inspect_dataset(dataset, split="train", sample_size=5):
    """
    Проверяет наличие пустых значений и валидность позиций ответов в датасете.

    Параметры:
        dataset (DatasetDict): словарь с train/validation/test.
        split (str): какую часть проверять.
        sample_size (int): сколько плохих примеров вывести.
    """
    bad_samples = []

    print(f"\n🔍 Проверка части '{split}'...")
    print(f"Всего примеров: {len(dataset[split])}")

    for i, item in enumerate(dataset[split]):
        context = item.get("context", "")
        question = item.get("question", "")
        answers = item.get("answers", {"text": [], "answer_start": []})

        if not context.strip() or not question.strip() or not answers["text"]:
            bad_samples.append((i, "Пустое поле"))
            continue

        for text, start in zip(answers["text"], answers["answer_start"]):
            if start >= len(context):
                bad_samples.append((i, "Начало ответа за пределами контекста"))
                break
            if text not in context:
                bad_samples.append((i, "Ответ не найден в контексте"))
                break

    print(f"⚠️ Найдено проблемных примеров: {len(bad_samples)}")

    if sample_size > 0 and bad_samples:
        print("\n📌 Примеры плохих данных:")
        for i, reason in bad_samples[:sample_size]:
            print(f"  👉 Пример {i}, причина: {reason}")
            print(f"     Вопрос: {dataset[split][i]['question']}")
            print(f"     Ответ: {dataset[split][i]['answers']}")
            print(f"     Контекст: {dataset[split][i]['context']}...\n")

    return bad_samples


inspect_dataset(dataset, split="train", sample_size=3)
inspect_dataset(dataset, split="validation", sample_size=3)
inspect_dataset(dataset, split="test", sample_size=3)


🔍 Проверка части 'train'...
Всего примеров: 45328
⚠️ Найдено проблемных примеров: 8050

📌 Примеры плохих данных:
  👉 Пример 116, причина: Ответ не найден в контексте
     Вопрос: Какой запрет, касательно моторов, действует в Формуле-1 по ходу сезона?
     Ответ: {'text': ['На доработку силовых установок'], 'answer_start': [-1]}
     Контекст: Хотя машины Формулы-1 нередко превышают скорость 300 км/ч, по абсолютной скорости Формула-1 не может считаться самой быстрой автогоночной серией, так как многие параметры моторов в ней существенно ограничены (ограничен объём, действует запрет на доработку силовых установок по ходу сезона, и т. п.). Тем не менее, по средней скорости на круге среди шоссейно-кольцевых автогонок (исключая т. н. овалы ) Формуле-1 нет равных. Это становится возможным благодаря очень эффективным тормозам и аэродинамике....

  👉 Пример 117, причина: Ответ не найден в контексте
     Вопрос: По какому критерию Формула-1 не может считаться самой быстрой?
     Ответ: {'text'

[]

## Пример данных из тренировочной выборки

In [17]:
import json


def print_pretty_sample(sample):
    formatted_sample = {
        "id": sample["id"],
        "title": sample["title"],
        "context": sample["context"],
        "question": sample["question"],
        "answer": {
            "text": sample["answers"]["text"][0],
            "answer_start": sample["answers"]["answer_start"][0]
        }
    }
    print(json.dumps(formatted_sample, indent=4, ensure_ascii=False))


sample = dataset["train"][0]
print("Пример точки данных из тренировочной выборки:")
print_pretty_sample(dataset["train"][0])
print(f"ID: {sample['id']}")
print(f"title: {sample['title']}")
print(f"context: {sample['context']}")
print(f"question: {sample['question']}")
print(f"answer: {sample['answers']['text']}")
print(f"answer_start: {sample['answers']['answer_start']}")

Пример точки данных из тренировочной выборки:
{
    "id": 62310,
    "title": "SberChallenge",
    "context": "В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.",
    "question": "чем представлены органические остатки?",
    "answer": {
        "text": "известковыми выделениями сине-зелёных водорослей",
        "answer_start": 109
    }
}
ID: 62310
title: SberChallenge
context: В протерозойских отложениях органическ

## Токенизация данных

In [11]:
tokenized_dataset = processor.get_tokenized_dataset(dataset)
print("Токенизированный датасет:", end = '\n\n')
print(tokenized_dataset)

Токенизированный датасет:

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'overflow_to_sample_mapping'],
        num_rows: 45544
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'overflow_to_sample_mapping'],
        num_rows: 5063
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'overflow_to_sample_mapping'],
        num_rows: 24022
    })
})


## Токенизированные данные

In [13]:
tokenized_sample = tokenized_dataset["train"][10]
print(tokenized_sample, end = '\n\n')
print(f"input_ids: {tokenized_sample['input_ids']}")
print(f"token_type_ids: {tokenized_sample['token_type_ids']}")
print(f"attention_mask: {tokenized_sample['attention_mask']}")
print(f"start_positions: {tokenized_sample['start_positions']}")
print(f"end_positions: {tokenized_sample['end_positions']}")
print(f"overflow_to_sample_mapping: {tokenized_sample['overflow_to_sample_mapping']}")

original_sample_idx = tokenized_sample["overflow_to_sample_mapping"]
original_sample = dataset["train"][10]
print(original_sample)

{'input_ids': [101, 781, 30134, 2077, 57308, 10807, 24639, 56028, 626, 102, 11642, 56028, 851, 89748, 56028, 6653, 25178, 9629, 56028, 128, 87265, 1916, 17814, 875, 15846, 1469, 6808, 2785, 95885, 1768, 132, 94909, 3364, 38860, 15274, 4929, 154, 17789, 8221, 845, 1574, 120, 15850, 24639, 16421, 56028, 6637, 16192, 11364, 6530, 901, 4929, 150, 128, 144, 17789, 8221, 845, 1574, 158, 6636, 4929, 138, 128, 146, 17789, 8221, 845, 1574, 9809, 86167, 1469, 50796, 15569, 38860, 122, 128, 1997, 6637, 144, 128, 140, 114, 1641, 13949, 18149, 62579, 1469, 6723, 1574, 132, 6776, 4105, 128, 1703, 20472, 18149, 3590, 845, 16367, 4857, 56028, 626, 17490, 18777, 79974, 842, 72639, 845, 16017, 138, 128, 4370, 17789, 8221, 120, 1516, 13629, 1469, 6723, 1574, 122, 132, 781, 12580, 15969, 89748, 851, 3106, 24006, 7173, 50217, 3590, 845, 3955, 128, 4370, 17789, 8221, 845, 1574, 132, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## Инициализация и обучение модели

In [14]:
trainer = QAModelTrainer(tokenizer=processor.tokenizer)
trainer.setup_trainer(tokenized_dataset, dataset)
train_history = trainer.train()

trainer.save_model()
print("Модель успешно обучена и сохранена!")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,1.562000,1.499309,0.000000,1.931493,100,0.000000,1.931493,100,0.000000,0.000000,1.931493,0.000000
2,1.253800,1.458453,0.000000,1.905556,100,0.000000,1.905556,100,0.000000,0.000000,1.905556,0.000000
3,0.882600,1.597729,0.000000,1.557828,100,0.000000,1.557828,100,0.000000,0.000000,1.557828,0.000000


Модель успешно обучена и сохранена!


## Пример использования модели

In [15]:
qa_system = QAPipeline()

context = "Автокодировщиком называется нейронная сеть, обученная пытаться скопировать свой вход в выход"
question = "Что такое автокодировщик?"

result = qa_system.predict(context, question)
print(f"Результирующий словарь: {result}")
print("Результат предсказания:")
print(f"Вопрос: {question}")
print(f"Ответ: {result['answer']}")
print(f"Точность: {result['score']:.2f}")

Device set to use cuda:0


Результирующий словарь: {'score': 0.8621006608009338, 'start': 28, 'end': 42, 'answer': 'нейронная сеть'}
Результат предсказания:
Вопрос: Что такое автокодировщик?
Ответ: нейронная сеть
Точность: 0.86


## Прогонка на тестовой выборке

In [17]:
print("📊 Результаты на тестовой выборке:")
test_data = dataset["test"]

for i, sample in enumerate(test_data.select(range(10))):
    context = sample["context"]
    question = sample["question"]
    true_answer = sample["answers"]["text"]

    prediction = qa_system.predict(context, question)

    print(f"Пример {i + 1}")
    print(f"Вопрос: {question}")
    print(f"Контекст: {context}")
    print(f"Правильный ответ: {true_answer}")
    print(f"Предсказание: {prediction['answer']}")
    print(f"Точность: {prediction['score']:.2f}", end = '\n\n')

📊 Результаты на тестовой выборке:
Пример 1
Вопрос: У каких организмов отсутствуют настоящие дифференцированные клетки?
Контекст: Многоклеточный организм — внесистематическая категория живых организмов, тело которых состоит из многих клеток, большая часть которых (кроме стволовых, например, клеток камбия у растений) дифференцированы, то есть различаются по строению и выполняемым функциям. Следует отличать многоклеточность и колониальность. У колониальных организмов отсутствуют настоящие дифференцированные клетки, а следовательно, и разделение тела на ткани. Граница между многоклеточностью и колониальностью нечёткая. Например, вольвокс часто относят к колониальным организмам, хотя в его колониях есть чёткое деление клеток на генеративные и соматические. Кроме дифференциации клеток, для многоклеточных характерен и более высокий уровень интеграции, чем для колониальных форм. Многоклеточные животные, возможно, появились на Земле 2,1 миллиарда лет назад, вскоре после кислородной революции .


## Оценка результата на тестовой выборке

In [27]:
print("📊 Вычисляем метрики на всей тестовой выборке...")

squad_metric = evaluate.load("squad_v2")

test_data = dataset["train"]

predictions = []
references = []

for sample in test_data.select(range(100)):
    context = sample["context"]
    question = sample["question"]
    true_answers = sample["answers"]["text"]
    
    result = qa_system.predict(context, question)
    
    predictions.append({
        "id": str(sample["id"]),
        "prediction_text": result["answer"],
        "no_answer_probability": 0.0
    })
    
    references.append({
        "id": str(sample["id"]),
        "answers": {
            "text": true_answers,
            "answer_start": sample["answers"]["answer_start"]
        }
    })

metrics = squad_metric.compute(predictions=predictions, references=references)

print("📈 Метрики качества на тестовой выборке:")
print(f"📌 Exact Match (EM): {metrics['exact']:.2f}")
print(f"📌 F1 Score: {metrics['f1']:.2f}")

📊 Вычисляем метрики на всей тестовой выборке...
📈 Метрики качества на тестовой выборке:
📌 Exact Match (EM): 86.00
📌 F1 Score: 95.45
